In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
#import seaborn as sns
import numpy as np
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.decomposition import PCA
from feature_engineering import *
from train_and_pred import *
from score import *


# Read File

In [2]:
data = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv') #Data Test for kaggle

### Replace values where NaN has meaning

In [3]:
data = replace_NaN_meaning(data)
data_test = replace_NaN_meaning(data_test)

In [4]:
data = adding_features(data)
data_test = adding_features(data_test)

## Transforming some numerical variables that are really categorical

In [5]:
data = transform_numerical_col_categorical(data)
data_test = transform_numerical_col_categorical(data_test)

### Split features and target

In [6]:
lis_drop_num_columns = ['SalePrice','Id']
X = data.drop(lis_drop_num_columns,axis=1)
Y = data[['SalePrice']]

id_test_kaggle = data_test[['Id']]
X_kaggle = data_test.drop('Id',axis=1)

# Handle Missing Data

In [7]:
X, num_columns, categ_columns = handle_missing_data(X)
X_kaggle, num_columns_kg, categ_columns_kg = handle_missing_data(X_kaggle)

## Concat Dataframes

In [8]:
concat_df = pd.concat([X,X_kaggle])

## Normalize numerical data

In [9]:
concat_df[num_columns.columns] = preprocessing.scale(concat_df[num_columns.columns])

## Using One-Hot encoding

In [10]:
#use one-hot encoding in categorical data
one_hot_encoding_all = pd.get_dummies(concat_df[categ_columns.columns])

### Join categorical and numerical columns again

In [11]:
X_final, X_final_kaggle = split_data_dataKaggle(X, X_kaggle, one_hot_encoding_all,num_columns)

## Split train and test

In [12]:
#Split train x test
X_train, X_test, y_train, y_test = train_test_split(X_final, Y, shuffle=True, random_state=42)

# Train and prediction

#### Random Florest

In [13]:
reg1, y_pred1 = train_pred_RandomFlorest(X_train, y_train, X_test, grid_search=False)

#### Gradient Boosting Regressor

In [ ]:
reg2, y_pred2 = train_pred_GradientBoostingRegressor(X_train, y_train, X_test, grid_search=False)

### Lasso

In [ ]:
reg3, y_pred3 = train_pred_Lasso(X_train, y_train, X_test, grid_search=False)

### XGboost

In [ ]:
reg4, y_pred4 = train_pred_XGboost(X_train, y_train, X_test, grid_search=False)

## Final Predictor to Combine the predictors

In [ ]:
# Train
list_alg = ['rf', 'gb','lasso']
list_predictors = [reg1, reg2, reg3]
reg_comb = train_comb_predictor(X_train, y_train, list_alg, list_predictors)

#Test
list_predictions = [y_pred1, y_pred2, y_pred3]
y_pred_ens = test_comb_predictor(reg_comb, list_alg, list_predictions)

In [ ]:
print(score(y_test,y_pred1))
print(score(y_test,y_pred2))
print(score(y_test,y_pred3))

## Score

In [ ]:
score(y_test,y_pred_ens)

# Test Kaggle

In [ ]:
#make sure there is no NaN values
X_final_kaggle[num_columns.columns] = X_final_kaggle[num_columns.columns].groupby(num_columns.columns, axis = 1).transform(lambda x: x.fillna(x.mean()))

In [ ]:
Y_kaggle_predictions = []
for reg in list_predictors:
    Y_kaggle_predictions.append(reg.predict(X_final_kaggle))

### Ensemble Predictor 

In [ ]:
Y_kaggle_ens = test_comb_predictor(reg_comb, list_alg, Y_kaggle_predictions)

In [ ]:
df_submission_kaggle = pd.DataFrame(Y_kaggle_ens, columns =['SalePrice']) 
df_submission_kaggle['Id'] = id_test_kaggle

df_submission_kaggle.to_csv('submission.csv', index=False)

df_submission_kaggle.shape